In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import interpolate
from numpy.linalg import solve
from scipy.integrate import quad
import math

In [ ]:
shape_pts = np.array([[-8,1],[-6,10],[-5,-7],[0,-5],[1,9],[2,-8],[3,28],[5,-23],[7,12],[9,1],[10,10],[12,15],[14,18],[15,10]])

In [ ]:
'''Generate the target function f'''

%matplotlib inline
def TargetFunction(lst):
    x = []
    y = []
    for i in range(len(lst)):
        x.append(lst[i][0])
        y.append(lst[i][1])
    f = plt.figure()
    f.set_figwidth(15)
    f.set_figheight(5)
    plt.step(x,y,where = 'mid')
    plt.step(x,y,'r*',where = 'mid', label = 'Shape Points')
    plt.legend(['Shape Points'])
    plt.xlabel('X')
    plt.ylabel('f(X)')
    plt.title('Target Function')
    plt.grid()
    plt.legend(loc ="upper right")
    return plt


f = TargetFunction(shape_pts)




In [ ]:
'''Collocation abscissa points'''


x_min = shape_pts[0][0]
x_max = shape_pts[-1][0]
m = 100
assert m >= len(shape_pts)

x = []
y = []
for i in range(len(shape_pts)):
    x.append(shape_pts[i][0])
    y.append(shape_pts[i][1])

u = interpolate.interp1d(x, y, kind='nearest',fill_value='array-like')
x_tilde_pts = np.linspace(x_min, x_max, m)

y_new = u(x_tilde_pts)


y_new = u(x_tilde_pts)
f = plt.figure()
f.set_figwidth(15)
f.set_figheight(5)
plt.step(x,y,where = 'mid')
plt.step(x,y,'r*',where = 'mid', label = 'Shape Points')
plt.grid()
plt.xlabel('X')
plt.ylabel('f(X)')
plt.title('Collocation Points on Target Function (m = '+str(m)+')')
plt.scatter(x_tilde_pts,y_new,c='#3396FF',marker='x',label='Collocation Points')
plt.legend()
plt.show()


# print(len(x_tilde_pts))


In [ ]:
shape_pts = np.array([[-8,1],
                      [-6,10],
                      [-5,-7],
                      [0,-5],
                      [1,9],
                      [2,-8],
                      [3,28],
                      [5,-23],
                      [7,12],
                      [9,1],
                      [10,10],
                      [12,15],
                      [14,18],
                      [15,10]])

m = 100
x_min = shape_pts[0][0]
x_max = shape_pts[-1][0]
x_tilde_pts = np.linspace(x_min, x_max, m)



wavelength = x_max - x_min
kappa = 2*np.pi/wavelength

N = 22 # number of pairs of sine/cosine  


# print(len(x_tilde_pts))


#### Here the A matrix is generateded using collocation points

In [ ]:
'''Build the basis function evaluation matrix and target function vector at the collocation points'''
wavelength = x_tilde_pts[-1] - x_tilde_pts[0]
Kappa=2*(np.pi)/wavelength
N=22

# FourierBasis Function
def FourierBasis(x_tilde_pts):
    A=np.zeros((len(x_tilde_pts),2*N+1))
    for i in range (len(x_tilde_pts)):
        for j in range((2*N+1)):
            if (j==0):
                A[i,j]=1
            elif (j%2)==1:
                A[i,j]=np.cos(((j//2)+1)*Kappa*x_tilde_pts[i])
            elif (j%2)==0:
                A[i,j]=np.sin((j//2)*Kappa*x_tilde_pts[i])
    return np.array(A)

#Plot Function
def Plot_fourierbasis(function,x_pts):
    f = plt.figure()
    f.set_figwidth(30)
    f.set_figheight(5)
    for i in range(len(function[0])):
        if i==0:
            plt.plot(x_pts,function[:,0],label='1')
        elif i%2==1:
            plt.plot(x_pts,function[:,i], label='cos(%ikx)'%(((i/2)+0.5)))
        elif i%2==0 and i!=0:
            plt.plot(x_pts,function[:,i], label='sin(%ikx)'%(i/2))
        for j in range(len(function[0])//2):
            plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
        plt.xlabel('x'); plt.ylabel('Cos(X),Sin(X)')
        plt.title('Fourier basis func at Mode =' + str(N))
    return plt.show()
    
A = FourierBasis(x_tilde_pts)
Plot_fourierbasis(A,x_tilde_pts)

# print(len(x_tilde_pts))


sin_function = []
for i in range(1,N):
    sine_function = [np.sin(kappa*i*x_tilde_pts),np.cos(kappa*i*x_tilde_pts)]
    sin_function.append(sine_function)

ones = np.ones(len(x_tilde_pts))

A = []

A.append(ones.tolist())



In [ ]:
'''Compute optimal coefficient vector'''
rank = np.linalg.matrix_rank(A)

if A.shape[0] > A.shape[1]:
    print('A is overdetermined.')
elif A.shape[0] < A.shape[1]:
    print('A is underdetermined.')  
else:
    print('A is determined.')

if np.linalg.matrix_rank(A) == min(A.shape):
    print('A is full rank.')
else:
    print('A is rank deficient.')

#--------------------------------------------------------------#

f_tilde_vec = np.array(y_new)

if np.linalg.matrix_rank(A) == min(A.shape):
    print('A is full rank; solve for least squares.')
    c_tilde_star_vec = solve(A.transpose()@A, A.transpose()@f_tilde_vec)
else:
    print('A is rank deficient; solve for shortest least squares')
    c_tilde_star_vec = universal_solve(A.transpose()@A, A.transpose()@f_tilde_vec,
                                       pivot_tol=1e-6)
    
print('')
print('c_tilde_star_vec:\n')
for j,c in enumerate(c_tilde_star_vec):
    if j == 0:
        print('a_%2i = %10.3e'%(j,c))
    elif j%2 == 0:
        k = j/2
        print('b_%2i = %10.3e'%(k,c))
    else:
        k = (j+1)/2
        print('a_%2i = %10.3e'%(k,c))
        
print('')        
print('||r^*||_2 = %10.3e'%(np.linalg.norm(A@c_tilde_star_vec-f_tilde_vec)))


# print(len(x_tilde_pts))


In [ ]:
'''Build the best approximant function'''

points_n = max(2*m, 200)
def bestg_vec_func(x_tilde_pts):
    matrics_a = FourierBasis(x_tilde_pts)
    return matrics_a@c_tilde_star_vec

x_tilde_pts_new = np.linspace(x_min, x_max, points_n)
bestg_vec_1 = bestg_vec_func(x_tilde_pts_new)


# print(len(x_tilde_pts))
# print(len(x_tilde_pts_new))

In [ ]:
'''Plot comparison of f and g_best_vec'''


f = plt.figure()
f.set_figwidth(15)
f.set_figheight(5)
plt.step(shape_pts[:,0], shape_pts[:,1], where='mid',label='f(x)', color='k')
plt.plot(shape_pts[:,0], shape_pts[:,1], '*', color='red',label='Shape points')
plt.grid()
plt.plot(x_tilde_pts_new,bestg_vec_1,'--',label='g(x)', color='b')
plt.legend( loc='upper right')
plt.xlabel('x')
plt.ylabel("f(x)/g(x)")
plt.show()


# print(len(x_tilde_pts))


In [ ]:
'''L2 norm error'''
def f_integrand(x):
    return u(x)**2
f_integral = quad(f_integrand, x_min, x_max, limit=1000)
f_l2=np.sqrt(f_integral)

def g2_integrand(x):
    g_vec = bestg_vec_func(np.array([x]))
    return g_vec**2

print('||f||_L2 =%10.6f'%f_l2[0])
print('quadrature error =%10.6e'%f_l2[1])

def g2_integrand(x):
    g_vec = bestg_vec_func(np.array([x]))
    return g_vec**2

print('')

g2_integral, error = quad(g2_integrand, x_min, x_max, limit=1000)
print('||g||_L2 =', math.sqrt(g2_integral))
print('quadrature error =', error)

def f_minus_g_2_integrand(x):
    f_x = u(np.array([x]))
    g_vec = bestg_vec_func(np.array([x]))
    return (f_x-g_vec)**2

print('')

f_minus_g_2_integral, error = quad(f_minus_g_2_integrand, x_min, x_max, limit=1000)
print('||f-g||_L2 =', math.sqrt(f_minus_g_2_integral))
print('quadrature error =', error)
r_err=(math.sqrt(f_minus_g_2_integral)/np.sqrt(f_integral)*100)
print('')
print('Relative error = %10.6f'%r_err[0],'%')


# print(len(x_tilde_pts))


In [ ]:
'''Residual of the best approximant'''

points_n = max(3*m, 500)
x_tilde_pts_new = np.linspace(x_min, x_max, points_n)

residual = u(x_tilde_pts_new) - bestg_vec_func(x_tilde_pts_new)
residual_collocation_pts = u(x_tilde_pts) - bestg_vec_func(x_tilde_pts)
print('Positives at collocation pts = ', tuple([True for p in residual_collocation_pts if p>0]).count(True))
print('Negatives at collocation pts = ', tuple([True for p in residual_collocation_pts if p<0]).count(True))
print('Positives = ', tuple([True for p in residual if p>0]).count(True))
print('Negatives = ', tuple([True for p in residual if p<0]).count(True))
  
f = plt.figure()
f.set_figwidth(15)
f.set_figheight(5)
plt.plot(x_tilde_pts_new, residual)
plt.plot(x_tilde_pts, u(x_tilde_pts)-bestg_vec_func(x_tilde_pts),'bx',label='collocation pts',color='red')
plt.title(r'Residual $r(x) :\!\!= f(x) - g(x)$ ($m\rightarrow\infty$'+str(m)+', n='+str(len(A[0]))+')', fontsize=20)

plt.ylabel(r'$r(x)$')
plt.xlabel(r'$x$')

plt.legend(loc='best')
plt.grid(True)
plt.show()


# print(len(x_tilde_pts))
# print(len(x_tilde_pts_new))

In [ ]:
'''Collocation abscissa points'''

m=200
x_tilde_pts = np.linspace(x_min,x_max,m)
u = interpolate.interp1d(shape_pts[:,0], shape_pts[:,1], kind='nearest')
y_new= u(x_tilde_pts)

f = plt.figure()
f.set_figwidth(15)
f.set_figheight(5)
plt.scatter(x_tilde_pts,y_new, marker = '*')
plt.step(shape_pts[:,0], shape_pts[:,1], where='mid')
plt.plot(shape_pts[:,0], shape_pts[:,1], '*', color='red',label= 'Shape point')
plt.legend()
plt.title('Collocation Points on Target Function (m='+str(m)+')')
plt.xlabel('x')
plt.ylabel('f(x)')
plt.grid()

# print(len(x_tilde_pts))
# print(len(x_tilde_pts_new))

In [ ]:
'''Build the basis function evaluation matrix and target function vector at the collocation points'''
A=FourierBasis(x_tilde_pts)
Plot_fourierbasis(A,x_tilde_pts)

# print(len(x_tilde_pts))
# print(len(x_tilde_pts_new))

In [ ]:
'''Compute optimal coefficient vector'''
rank = np.linalg.matrix_rank(A)

if A.shape[0] > A.shape[1]:
    print('A is overdetermined.')
elif A.shape[0] < A.shape[1]:
    print('A is underdetermined.')  
else:
    print('A is determined.')

if np.linalg.matrix_rank(A) == min(A.shape):
    print('A is full rank.')
else:
    print('A is rank deficient.')

##########################################################################################

f_tilde_vec = np.array(y_new)

if np.linalg.matrix_rank(A) == min(A.shape):
    print('A is full rank; solve for least squares.')
    c_tilde_star_vec = solve(A.transpose()@A, A.transpose()@f_tilde_vec)
else:
    print('A is rank deficient; solve for shortest least squares')
    c_tilde_star_vec = universal_solve(A.transpose()@A, A.transpose()@f_tilde_vec,
                                       pivot_tol=1e-6)
    
print('')
print('c_tilde_star_vec:\n')
for j,c in enumerate(c_tilde_star_vec):
    if j == 0:
        print('a_%2i = %10.3e'%(j,c))
    elif j%2 == 0:
        k = j/2
        print('b_%2i = %10.3e'%(k,c))
    else:
        k = (j+1)/2
        print('a_%2i = %10.3e'%(k,c))
        
print('')        
print('||r^*||_2 = %10.3e'%(np.linalg.norm(A@c_tilde_star_vec-f_tilde_vec)))

# print(len(x_tilde_pts))
# print(len(x_tilde_pts_new))

In [ ]:
'''Build the best approximant function'''
points_n = max(2*m, 200)

x_tilde_pts_new = np.linspace(x_min, x_max, points_n)
bestg_vec_1 = bestg_vec_func(x_tilde_pts_new)


# print(len(x_tilde_pts))
# print(len(x_tilde_pts_new))

In [ ]:
'''Plot comparison of f and g_best_vec'''


f = plt.figure()
f.set_figwidth(15)
f.set_figheight(5)
plt.step(shape_pts[:,0], shape_pts[:,1], where='mid',label='f(x)', color='k')
plt.plot(shape_pts[:,0], shape_pts[:,1], '*', color='red',label='Shape points')
plt.grid()
plt.plot(x_tilde_pts_new,bestg_vec_1,'--',label='g(x)', color='b')
plt.legend( loc='upper right')
plt.xlabel('x')
plt.ylabel("f(x)/g(x)")
plt.show()

# print(len(x_tilde_pts))
# print(len(x_tilde_pts_new))

In [ ]:
'''L2 norm error'''


g2_integral, error = quad(g2_integrand, x_min, x_max, limit=1000)
print('||g||_L2 =', math.sqrt(g2_integral))
print('quadrature error =', error, '\n')



f_minus_g_2_integral, error = quad(f_minus_g_2_integrand, x_min, x_max, limit=1000)
print('||f-g||_L2 =', math.sqrt(f_minus_g_2_integral))
print('quadrature error =', error)
r_err=(math.sqrt(f_minus_g_2_integral)/np.sqrt(f_integral)*100)
print('\n')
print('Relative error = %10.6f'%r_err[0],'%')

# print(len(x_tilde_pts))
# print(len(x_tilde_pts_new))

In [ ]:
'''Residual of the best approximant'''

points_n = max(3*m, 500)
x_tilde_pts_new = np.linspace(x_min, x_max, points_n)

residual = u(x_tilde_pts_new) - bestg_vec_func(x_tilde_pts_new)
residual_collocation_pts = u(x_tilde_pts) - bestg_vec_func(x_tilde_pts)
print('Positives at collocation pts = ', tuple([True for p in residual_collocation_pts if p>0]).count(True))
print('Negatives at collocation pts = ', tuple([True for p in residual_collocation_pts if p<0]).count(True))
print('Positives = ', tuple([True for p in residual if p>0]).count(True))
print('Negatives = ', tuple([True for p in residual if p<0]).count(True))
  
f = plt.figure()
f.set_figwidth(15)
f.set_figheight(5)
plt.plot(x_tilde_pts_new, residual)
plt.plot(x_tilde_pts, u(x_tilde_pts)-bestg_vec_func(x_tilde_pts),'bx',label='collocation pts',color='red')
plt.title(r'Residual $r(x) :\!\!= f(x) - g(x)$ ($m\rightarrow\infty$'+str(m)+', n='+str(len(A[0]))+')', fontsize=20)

plt.ylabel(r'$r(x)$')
plt.xlabel(r'$x$')

plt.legend(loc='best')
plt.grid(True)
plt.show()

# print(len(x_tilde_pts))
# print(len(x_tilde_pts_new))

In [ ]:
print(len(x_tilde_pts))
print(len(x_tilde_pts_new))